# Proyecto de Optimización
## CIMAT Monterrey
### Aplicación del TSP al GTFS de Ciudad de México para la Planificación Eficiente de Rutas
- Semiramis G. de la Cruz
- Federico Salinas Samaniego

## Lectura de datos

In [2]:
import os
import pandas as pd

PATH = "../"
files = os.listdir(os.path.join(PATH, 'data'))
files

['agency.txt',
 'calendar.txt',
 'frequencies.txt',
 'routes.txt',
 'shapes.txt',
 'stops.txt',
 'stop_times.txt',
 'trips.txt']

In [18]:
df_agency = pd.read_csv(os.path.join(PATH, 'data', files[0]))
df_calendar = pd.read_csv(os.path.join(PATH, 'data', files[1]))
df_frequencies = pd.read_csv(os.path.join(PATH, 'data', files[2]))
df_routes = pd.read_csv(os.path.join(PATH, 'data', files[3]))
df_shapes = pd.read_csv(os.path.join(PATH, 'data', files[4]))
df_stops = pd.read_csv(os.path.join(PATH, 'data', files[5]))
df_stop_times = pd.read_csv(os.path.join(PATH, 'data', files[6]))
df_trips = pd.read_csv(os.path.join(PATH, 'data', files[7]))

In [13]:
df_agency.sample(5)

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
2,METRO,Sistema de Transporte Colectivo Metro,https://www.metro.cdmx.gob.mx/,America/Mexico_City,ES
1,CC,Corredores Concesionados,https://datos.cdmx.gob.mx/dataset/?q=corredore...,America/Mexico_City,ES
0,PUMABUS,Pumabús,https://www.dgsgm.unam.mx/pumabus,America/Mexico_City,ES
4,TL,Servicio de Tren Ligero,https://www.ste.cdmx.gob.mx/tren-ligero,America/Mexico_City,ES
3,MB,Metrobús,https://www.metrobus.cdmx.gob.mx,America/Mexico_City,ES


In [14]:
df_calendar.sample(5)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
6,0,1,1,1,1,1,1,1,20230101,20241231
7,3,0,0,0,0,0,0,1,20230101,20241231
0,9,0,0,0,1,0,0,1,20230101,20241231
4,5,0,0,0,0,0,1,1,20230101,20241231
3,6,1,1,1,0,1,0,0,20230101,20241231


In [15]:
df_frequencies.sample(5)

,trip_id,start_time,end_time,headway_secs,exact_times
1427,051110C000_0,5:00:00,19:00:00,1980,0
452,011010A000_1,7:00:00,24:00:00,240,0
699,055107B000_0,5:30:00,19:00:00,1080,0
1108,055161F000_1,18:00:00,20:00:00,2340,0
1357,0550119000_0,5:30:00,18:30:00,1680,0


In [16]:
df_routes.sample(5)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
137,CMX01017B,CC,17B,Xochimilco - Izazaga por Miramontes,3,9B26B6,FFFFFF
159,CMX0200L1,METRO,1,Pantitlán - Observatorio,1,F94F8E,FFFFFF
38,CMX0900R5,PUMABUS,PUMA5,Metro Universidad - Av. Universidad,3,00B3EE,NaN
191,CMX050124,RTP,124,Puerta Grande - Metro Mixcoac,3,A5C731,FFFFFF
172,CMX0511AN,RTP,11-A,Aragón por Av. 604 - Metro Chapultepec (Nochebús),3,E20613,FFFFFF


In [19]:
df_shapes.sample(5)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
103361,SH050104000_1,19.522288,-99.142140,108,NaN
5981,SH01010C000_0,19.447209,-99.102545,45,2.74
45881,SH09xvdf00R4000_1,19.322973,-99.183997,20,1.11
93022,SH05081A000_0,19.322704,-99.133876,225,NaN
21116,SH01017B000_0,19.271912,-99.131131,25,3.25


In [20]:
df_stop_times.sample(5)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,timepoint
18702,04200L5000_0,00:32:52,00:32:52,0400L5-SANJUAN0,26,0
21112,0510043000_1,01:04:22,01:04:22,0500431-JLFPTOMAZATLA,37,0
23074,0510141000_1,01:12:00,01:12:00,0501411-PUEBLAPTE,32,0
7665,010020A000_0,00:21:56,00:21:56,TRIOX0-DIPLOMATICOS,11,0
26005,051111A000_0,01:25:14,01:25:14,05145A0-CALZTLALNORM,34,0


In [21]:
df_stops.sample(5)

,stop_id,stop_name,stop_lat,stop_lon,zone_id,wheelchair_boarding
945,010C1100-CANEBUENOSA,Lago Caneguín - Lago de Buenos Aires,19.45513,-99.20350,010C1100-CANEBUENOSA,2
3101,010C1400-CANALJUANSUR,Canal de San Juan y Sur 16A,19.39197,-99.05950,010C1400-CANALJUANSUR,2
6925,0501460-LASMARGARITA,Las Margaritas,19.20096,-99.14899,0501460-LASMARGARITA,0
6086,0501661-REVCDAREVO,Av. Revolución - Cerrada Revolución,19.34174,-98.99458,0501661-REVCDAREVO,0
2597,010C1301-EJNALANAFRAN,Av. Ejército Nacional y Anatole France,19.43803,-99.19733,010C1301-EJNALANAFRAN,2


In [22]:
df_trips.sample(5)

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,shape_id
953,CMX0300L4,2,03200L4003_0,Alameda Oriente,Hidalgo to Alameda Oriente,0,SH0300L4003_0
88,CMX01010D,1,011010D000_1,Metro Camarones - Peñón,Metro Camarones to Peñón,1,SH01010D000_1
297,CMX0700L1,3,07300L1000_0,Cuautitlán,Buenavista to Cuautitlán,0,SH0700L1000_0
467,CMX050145,5,0550145000_0,Vicente Guerrero/José María Morelos,145 11:22 to Vicente Guerrero/José María Morelos,0,SH050145000_0
559,CMX05017E,5,055017E000_0,Cetram Universidad anden B,17-E 15:19 to Cetram Universidad anden B,0,SH05017E000_0
